# Amplitude Calibration with Error Amplification

### Prerequisites
This guide assumes you have a configured `DeviceSetup` as well as `Qubit` objects with assigned parameters. Please see [Getting Started tutorial](../../../../tutorials/sources/getting_started.ipynb) if you need to create your setup and qubits for the first time. 

You can run this notebook on real hardware in the lab. However, if you don't have the hardware at your disposal, you can also run the notebook "as is" using an emulated session (see below). 

If you are just getting started with the LabOne Q Applications Library, please don't hesitate to reach out to us at info@zhinst.com.

### Background
In this how-to guide, you'll optimize the pulse amplitude of the pi-pulse for a given qubit transition using error amplfication: the `amplitude_fine` experiment included in the LabOne Q Applications Library. It is recommended to have the pulse amplitude already roughly calibrated, e.g. with the `amplitude_rabi` experiment.

The amplitude of a pulse can be precisely calibrated using error amplifying gate sequences. These gate sequences apply the same quantum operation (qop) a variable number of times. Therefore, if each qop has a small error $\theta$ in the rotation angle then a sequence of multiple qop's will have a rotation error of $\text{reps}\cdot \theta$, where $\text{reps}$ is the number of repetitions. By implementing sequences with increasing number of repetitions we can obtain a correction value for the imperfect pulse amplitude (see also https://qiskit-community.github.io/qiskit-experiments/tutorials/calibrations). A larger number of repetitions leads to a higher precision in the pulse calibration, finally limited by the coherence time of the qubit.  

![](../../../images/amplitude_fine.svg "Demonstration of pulses and the corresponding rotation around the Bloch sphere during an amplitude fine experiment.")

In the `amplitude_fine` experiment, we add an x90 gate before the gates that are repeated, in order to move the ideal population to the equator of the Bloch sphere where the sensitivity to over/under rotations is the highest. 

![](../../../images/amplitude_fine_underrotation.svg "Plots showing the results of pulses with too-small amplitudes, which cause under-rotation, and those with correct amplitudes.")

Below we show the results of an amplitude-fine experiment for calibration an x180 operation. The ping-pong pattern in the figure on the left indicates an under-rotation which makes the initial state rotate less than $\pi$. The rotated phase per gate $\phi_{\text{x180}}$ is given by the frequency of the fitted oscillation. The comparison to the ideal angle defines the correction factor $c$

$c=\frac{\phi_{\text{x180}}}{\pi}$

Finally, we scale our amplitude by $c^{-1}$ to obtain the optimized amplitude. 



### Imports

You'll start by importing the amplitude_fine experiment from `laboneq_applications`, as well as `laboneq.simple` and a demo quantum platform containing a qpu and device setup to run in emulation mode. See the [Getting Started tutorial](../../../../tutorials/sources/getting_started.ipynb) for more details about the quantum platform and how to create your experimental setup.

In [ ]:
import numpy as np
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation
from laboneq.simple import *

from laboneq_applications.experiments import amplitude_fine
from laboneq_applications.qpu_types.tunable_transmon.demo_qpus import demo_platform

### QPU and Device Setup

You'll generate six qubits with pre-defined parameters, as well as a `Device_Setup` consisting of a SHFQC+, HDAWG, and PQSC. If you already have your own `DeviceSetup` and qubits configured, you'll instead initialize the session using your setup as shown in the [Getting Started tutorial](../../../../tutorials/sources/getting_started.ipynb).

In [ ]:
my_platform = demo_platform(6)

Then, you'll connect to the `Session`. Here we connect to an emulated one:

In [ ]:
session = Session(my_platform.setup)
session.connect(do_emulation=True)

### Create a `FolderStore` for Saving Data

The experiment `Workflows` can automatically save the inputs and outputs of all their tasks to the folder path we specify when instantiating the `FolderStore`. Here, we choose the current working directory.

In [ ]:
# import FolderStore from the `workflow` namespace of LabOne Q, which was imported
# from `laboneq.simple`
from pathlib import Path

folder_store = workflow.logbook.FolderStore(Path.cwd())

We disable saving in this guide. To enable it, simply run `folder_store.activate()`.

In [ ]:
folder_store.deactivate()

### Running the Amplitude Fine x180 Workflow

In the `amplitude_fine` experiment for an x180 gate, we apply the quantum operation `x180` multiple times, defined by the `repetitions`.

You'll now make the experiment workflow and run. For more details on what experiment workflows are and what tasks they execute, see the [Experiment Workflows tutorial](../../../../tutorials/sources/experiment_workflows.ipynb).

Let's first create the options class for the amplitude-fine experiment and inspect it using the tree view of the option fields per task:

In [ ]:
options = amplitude_fine.experiment_workflow_x180.options()
options

Or, alternatively, using the `show_fields` function from the `workflow` namespace of LabOne Q, which was imported from `laboneq.simple`:

In [ ]:
workflow.show_fields(options)

Notice that, unless we change it:

- the experiment is run in `AcquisitionType.INTEGRATION` and `AveragingMode.CYCLIC`, using 1024 averages (`count`)
- the experiment is run on the qubit $|g\rangle \leftrightarrow |e\rangle$ (`transition`)
- calibration traces are added (`use_cal_traces=True`) for the states $|g\rangle$ and $|e\rangle$ (`cal_states`)
- the analysis workflow will run automatically (`do_analysis=True`)
- the figures produced by the analysis are automatically closed (`close_figures=False`)
- the qubit parameters will not be updated (`update=False`)

Here, let's disable closing the figures:

In [ ]:
options.close_figures(False)

Now we run the experiment workflow on the first two qubits in parallel.

In [ ]:
# our qubits live here in the demo setup:
qubits = my_platform.qpu.qubits

exp_workflow = amplitude_fine.experiment_workflow_x180(
    session=session,
    qpu=my_platform.qpu,
    qubits=[qubits[0], qubits[1]],
    repetitions=[[1,2,3,4,5], [1,2,3,4,5]],
    options=options
)

workflow_results = exp_workflow.run()

#### Inspect the Tasks That Were Run

In [ ]:
for t in workflow_results.tasks:
    print(t)

#### Inspect the Output Simulation

You can also inspect the compiled experiment and plot the simulated output:

In [ ]:
compiled_experiment = workflow_results.tasks["compile_experiment"].output
plot_simulation(compiled_experiment, length=50e-6)

#### Inspecting the Source Code of the Pulse-Sequence Creation Task

You can inspect the source code of the `create_experiment` task defined in `amplitude_fine` to see how the experiment pulse sequence is created using quantum operations. To learn more about the latter, see the Quantum Operations tutorial (add link).

In [ ]:
amplitude_fine.create_experiment.src

To learn more about how to work with experiment `Workflows`, check out the [Experiment Workflows tutorial](../../../../tutorials/sources/experiment_workflows.ipynb).

Here, let's briefly inspect the analysis-workflow results.

#### Analysis Results

Let's check what tasks were run as part of the analysis workflow:

In [ ]:
analysis_workflow_results = workflow_results.tasks["analysis_workflow"]
for t in analysis_workflow_results.tasks:
    print(t)

We can access the qubit parameters extracted by the analysis from the output of the analysis-workflow:

In [ ]:
from pprint import pprint

pprint(analysis_workflow_results.output)  # noqa: T203

Check out the [Experiment Workflows tutorial](../../../../tutorials/sources/experiment_workflows.ipynb) to see how you can manually update the qubit parameters to these new values, or reset them to the old ones. 

### Running the Amplitude Fine x90 Workflow

We can use the same concept to optimize other quantum operations, e.g. the amplitude of the `x90` quantum operation.

In [ ]:
# our qubits live here in the demo setup:
qubits = my_platform.qpu.qubits

exp_workflow_x90 = amplitude_fine.experiment_workflow_x90(
    session=session,
    qpu=my_platform.qpu,
    qubits=[qubits[0], qubits[1]],
    repetitions=[[1,2,3,4,5], [1,2,3,4,5]],
)

workflow_results_x90 = exp_workflow_x90.run()

Note that `repetitions` is now in steps of two: As mentioned in the introduction, the sensitivity to over/under rotations is the highest for a final population at the equator of the Bloch sphere. 

### Running the general Amplitude Fine Workflow

Besides standard operations, we can use the same concept to optimize quantum operations in general. Define your own quantum operation and measure deviations from the ideal `target_angle` using the `amplitude_fine` experiment. 

In [ ]:
# our qubits live here in the demo setup:
qubits = my_platform.qpu.qubits
my_qop = "y90"

exp_workflow_general = amplitude_fine.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubits=[qubits[0], qubits[1]],
    amplification_qop=my_qop,
    target_angle=np.pi/2,
    phase_offset=-np.pi/2,
    repetitions=[[1,2,3,4], [1,2,3,4]],
)

workflow_results_general = exp_workflow_general.run()

Great! You've now run your amplitude-fine experiment. Check out other experiments in this manual to keep characterizing your qubits.